# Estadística aplicada

Actividad 6a - 27 de septiembre

Oscar Gabriel Martínez Rodríguez - 1962702

Grupo 43

In [25]:
## librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
## librerías específicas
from lifelines import CoxPHFitter, ExponentialFitter, WeibullFitter, LogNormalFitter
import seaborn as sns

In [27]:
##cargar datos
data = pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv')
data.head(5)

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo
1,8.0,8.6,5,262,6,0,1,0,ventas,medio
2,1.1,8.8,7,272,4,0,1,0,ventas,medio
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo


In [28]:
data.describe()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,6.128335,7.161017,3.803054,201.050337,3.498233,0.144610,0.238083,0.021268
std,2.486307,1.711691,1.232592,49.943099,1.460136,0.351719,0.425924,0.144281
min,0.900000,3.600000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,4.400000,5.600000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,6.400000,7.200000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,8.200000,8.700000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,10.000000,10.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


![Alt text](image.png)

#### depuración de base de datos

In [29]:
## verificar missing values
data.isnull().sum()

satisfaccion           0
evaluacion             0
proyectos              0
promedio_horas_mes     0
semestre               0
accidente              0
renunciar              0
ascenso_5years         0
departamento          26
salario                0
dtype: int64

In [30]:
## vistazo a la columna departamento
data['departamento']

0         ventas
1         ventas
2         ventas
3         ventas
4         ventas
          ...   
14994    soporte
14995    soporte
14996    soporte
14997    soporte
14998    soporte
Name: departamento, Length: 14999, dtype: object

Dado que 'departamento' es una variable categórica, imputar los missing values con algna métrica como el promedio no puede ser posible, al igual que con la moda podría sesgar los resultados y hacerlos menos representativos.  En este caso, lo más óptimo será eliminar los registros con valores nulos en el departamento.

In [31]:
data = data.dropna()
data.isna().sum()

satisfaccion          0
evaluacion            0
proyectos             0
promedio_horas_mes    0
semestre              0
accidente             0
renunciar             0
ascenso_5years        0
departamento          0
salario               0
dtype: int64

In [32]:
## verificar que no haya valores duplicados
data.duplicated().sum()

2982

Dado que hay 2982 valores duplicados, se proceden a eliminarlos y comprobar nuevamente.

In [33]:
data = data.drop_duplicates(keep='first').reset_index(drop=True)
data.duplicated().sum()

0

In [34]:
## verificar que todas las variables sean numéricas
data.dtypes

satisfaccion          float64
evaluacion            float64
proyectos               int64
promedio_horas_mes      int64
semestre                int64
accidente               int64
renunciar               int64
ascenso_5years          int64
departamento           object
salario                object
dtype: object

In [35]:
## convertir en variables numéricas las variables de departamento y salario
unique_dep = data['departamento'].unique()
unique_dep

array(['ventas', 'contabilidad', 'recursos humanos', 'mantenimiento',
       'soporte', 'administración', 'mercadotecnia', 'informática',
       'desarrollo de producto', 'Investigación'], dtype=object)

In [36]:
unique_sal = data['salario'].unique()
unique_sal

array(['bajo', 'medio', 'alto'], dtype=object)

In [37]:
## obtener dummies de departamento
dummies_dep = pd.get_dummies(data['departamento'], prefix = 'dep').astype(int)
dummies_dep.head()

,dep_Investigación,dep_administración,dep_contabilidad,dep_desarrollo de producto,dep_informática,dep_mantenimiento,dep_mercadotecnia,dep_recursos humanos,dep_soporte,dep_ventas
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1


In [39]:
#quedarse con n-1 dummies, eliminando el último departamento (ventas)
dummies_dep = dummies_dep.drop(columns= 'dep_ventas')
dummies_dep.head(5)

,dep_Investigación,dep_administración,dep_contabilidad,dep_desarrollo de producto,dep_informática,dep_mantenimiento,dep_mercadotecnia,dep_recursos humanos,dep_soporte
0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0


In [42]:
## obtener dummies de salario
dummies_sal = pd.get_dummies(data['salario'], prefix = 'sal').astype(int)
dummies_sal.head(5)

,sal_alto,sal_bajo,sal_medio
0,0,1,0
1,0,0,1
2,0,0,1
3,0,1,0
4,0,1,0


In [43]:
##quedarse con n-1 dummies, eliminando el salario alto
dummies_sal = dummies_sal[['sal_bajo', 'sal_medio']]
dummies_sal.head(5)

,sal_bajo,sal_medio
0,1,0
1,0,1
2,0,1
3,1,0
4,1,0


In [44]:
## concatenar los dummies
data = pd.concat([data, dummies_dep, dummies_sal], axis = 1)
data.head(5)

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario,...,dep_administración,dep_contabilidad,dep_desarrollo de producto,dep_informática,dep_mantenimiento,dep_mercadotecnia,dep_recursos humanos,dep_soporte,sal_bajo,sal_medio
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo,...,0,0,0,0,0,0,0,0,1,0
1,8.0,8.6,5,262,6,0,1,0,ventas,medio,...,0,0,0,0,0,0,0,0,0,1
2,1.1,8.8,7,272,4,0,1,0,ventas,medio,...,0,0,0,0,0,0,0,0,0,1
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo,...,0,0,0,0,0,0,0,0,1,0
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo,...,0,0,0,0,0,0,0,0,1,0


In [50]:
data = data.drop(columns=['departamento', 'salario'])

In [51]:
## calcular estadísticas descriptivas
data.describe()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,dep_Investigación,dep_administración,dep_contabilidad,dep_desarrollo de producto,dep_informática,dep_mantenimiento,dep_mercadotecnia,dep_recursos humanos,dep_soporte,sal_bajo,sal_medio
count,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.00000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000
mean,6.296581,7.166825,3.802852,200.473522,3.364857,0.154282,0.166041,0.016929,0.057877,0.036361,0.051789,0.057210,0.081394,0.18714,0.056125,0.050121,0.151864,0.478692,0.438746
std,2.410700,1.683426,1.163238,48.727813,1.330240,0.361234,0.372133,0.129012,0.233520,0.187194,0.221610,0.232252,0.273451,0.39004,0.230173,0.218204,0.358904,0.499567,0.496254
min,0.900000,3.600000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.800000,5.700000,3.000000,157.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.600000,7.200000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.200000,8.600000,5.000000,243.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,1.000000
max,10.000000,10.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


Tras limpiar la base de datos, eliminando primero los registros con un valor nulo, y luego eliminando duplicados, se quedaron 11991 registros.

Un 16.60% de los registros renunciaron.  Los empleados en promedio trabajan 3.36 semestres, y dentro de este periodo un 15.42% sufrió un accidente dentro de la empresa

#### Estimar el modelo de Cox

In [53]:
cox1 = CoxPHFitter().fit(data, 'semestre', 'renunciar')
cox1.print_summary()

<lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
      baseline estimation = breslow
   number of observations = 11991
number of events observed = 1991
   partial log-likelihood = -15884.05
         time fit was run = 2023-09-24 06:22:08 UTC

---
                             coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                          
satisfaccion                -0.24       0.78       0.01            -0.26            -0.22                 0.77                 0.80
evaluacion                  -0.01       0.99       0.01            -0.04             0.02                 0.96                 1.02
proyectos                   -0.29       0.75       0.02            -0.33            -0.25                 0.72                 0.78
promedio_horas_mes           0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
accidente                   -1.25       0.29       0.10            -1.44            -1.05                 0.24                 0.35
ascenso_5years              -1.38       0.25       0.36            -2.08            -0.68                 0.13                 0.51
dep_Investigación           -0.35       0.70       0.12            -0.58            -0.12                 0.56                 0.89
dep_administración          -0.13       0.88       0.15            -0.42             0.16                 0.66                 1.17
dep_contabilidad            -0.11       0.90       0.10            -0.32             0.10                 0.73                 1.10
dep_desarrollo de producto   0.01       1.01       0.10            -0.20             0.21                 0.82                 1.24
dep_informática             -0.06       0.94       0.09            -0.24             0.12                 0.79                 1.13
dep_mantenimiento            0.08       1.08       0.07            -0.05             0.21                 0.95                 1.24
dep_mercadotecnia           -0.01       0.99       0.10            -0.22             0.19                 0.81                 1.21
dep_recursos humanos         0.08       1.09       0.10            -0.12             0.29                 0.89                 1.33
dep_soporte                  0.10       1.11       0.07            -0.04             0.24                 0.96                 1.27
sal_bajo                     1.54       4.66       0.15             1.25             1.83                 3.48                 6.23
sal_medio                    1.15       3.15       0.15             0.85             1.44                 2.35                 4.22

                             cmp to      z      p   -log2(p)
covariate                                                   
satisfaccion                   0.00 -26.89 <0.005     526.58
evaluacion                     0.00  -0.76   0.44       1.17
proyectos                      0.00 -14.75 <0.005     161.15
promedio_horas_mes             0.00   3.88 <0.005      13.25
accidente                      0.00 -12.41 <0.005     115.04
ascenso_5years                 0.00  -3.88 <0.005      13.21
dep_Investigación              0.00  -3.01 <0.005       8.56
dep_administración             0.00  -0.89   0.37       1.42
dep_contabilidad               0.00  -1.05   0.29       1.77
dep_desarrollo de producto     0.00   0.09   0.93       0.11
dep_informática                0.00  -0.65   0.52       0.95
dep_mantenimiento              0.00   1.23   0.22       2.18
dep_mercadotecnia              0.00  -0.11   0.91       0.14
dep_recursos humanos           0.00   0.81   0.42       1.26
dep_soporte                    0.00   1.42   0.16       2.67
sal_bajo                       0.00  10.36 <0.005      81.15
sal_medio          

In [55]:
## ver coeficientes sin redondear
cox1.params_

covariate
satisfaccion                 -0.242337
evaluacion                   -0.010687
proyectos                    -0.291606
promedio_horas_mes            0.001839
accidente                    -1.246686
ascenso_5years               -1.380084
dep_Investigación            -0.350547
dep_administración           -0.130262
dep_contabilidad             -0.110158
dep_desarrollo de producto    0.009468
dep_informática              -0.058469
dep_mantenimiento             0.081424
dep_mercadotecnia            -0.011698
dep_recursos humanos          0.083791
dep_soporte                   0.100538
sal_bajo                      1.538901
sal_medio                     1.147612
Name: coef, dtype: float64

In [56]:
## ver los exp(coef) sin redondear
cox1.hazard_ratios_

covariate
satisfaccion                  0.784792
evaluacion                    0.989370
proyectos                     0.747063
promedio_horas_mes            1.001841
accidente                     0.287456
ascenso_5years                0.251557
dep_Investigación             0.704303
dep_administración            0.877865
dep_contabilidad              0.895692
dep_desarrollo de producto    1.009513
dep_informática               0.943208
dep_mantenimiento             1.084830
dep_mercadotecnia             0.988370
dep_recursos humanos          1.087401
dep_soporte                   1.105766
sal_bajo                      4.659468
sal_medio                     3.150662
Name: exp(coef), dtype: float64

##### Interpretación de coeficientes

Comenzando por establecer las hipótesis:

$H_o$: La variable no afecta

$H_a$: La variable tiene un efecto sobre que los empleados renuncien 

Estableciendo un valor alfa = 0.05

Las variables que no tienen algún efecto sobre que los empleados renuncien son:

1. **evaluación** (P-value = 0.44 > alfa = 0.05)
2. **dep_administración** (P-value = 0.37 > alfa = 0.05)
3. **dep_contabilidad** (P-value = 0.29 > alfa = 0.05)
4. **dep_desarrollo de producto** (P-value = 0.93 > alfa = 0.05)
5. **dep_informática** (P-value = 0.52 > alfa = 0.05)
6. **dep_mantenimiento** (P-value = 0.22 > alfa = 0.05)
7. **dep_mercadotecnia** (P-value = 0.91 > alfa = 0.05)
8. **dep_recursos humanos** (P-value = 0.42 > alfa = 0.05)
9. **dep_soporte** (P-value = 0.16 > alfa = 0.05)

Ahora, comenzando para aquellas que sí son significativas y tienen efecto sobre que los empleados renuncien:

**satisfaccion**  (P-value < 0.005 < alfa = 0.05)

Dado que el coef = -0.242337 < 0, se interpreta que una calificación menor de satisfacción en los empleados aumenta el riesgo que renuncien.

In [60]:
p = (1- cox1.hazard_ratios_['satisfaccion']).round(4)
p

0.2152

El riesgo que los empleados renuncien se reduce en un 21.52% por cada grado más de calificación de satisfacción.

**proyectos** (P-value < 0.005 < alfa = 0.05)

Dado que el coef = -0.291606 < 0, se interpreta que a menor número de proyectos asignados al empleado aumenta el riesgo que renuncien.

In [62]:
p = (1- cox1.hazard_ratios_['proyectos']).round(4)
p

0.2529

El riesgo que los empleados renuncien se reduce en un 25.29% por cada proyecto asignado al empleado.

**promedio_horas_mes** (P-value < 0.005 < alfa = 0.05)

Dado que el coef = 0.001839 > 0, un promedio mayor de horas mensuales trabajadas aumenta el riesgo de renunciar de parte de los empleados.

In [63]:
p = (cox1.hazard_ratios_['promedio_horas_mes'] - 1).round(4)
p

0.0018

El riesgo que los empleados renuncien aumenta en un 0.18% por cada hora mensual promedio más trabajada.

**accidente** (P-value < 0.005 < alfa = 0.05)

Dado que el coef = -1.246686, se interpreta que el haber sufrido accidentes dentro de la empresa reduce el riesgo de renunciar.

In [64]:
p = (1-cox1.hazard_ratios_['accidente']).round(4)
p

0.7125

El riesgo que los empleados renuncien se reduce en un 71.25% al haber sufrido un accidente dentro de la empresa comparado a no haber sufrido alguno.

**ascenso_5years** (P-value < 0.005 < alfa = 0.05)

Dado que el coef = -1.380084 < 0, se interpreta que el no haber ascendido durante los últimos 5 años aumenta el riesgo de renunciar.

In [65]:
p = (1- cox1.hazard_ratios_['ascenso_5years']).round(4)
p

0.7484

El riesgo de que los empleados renuncien se reduce en un 74.84% al haber sido ascendido durante los últimos 5 años.

**dep_Investigación** (P-value < 0.005 < alfa = 0.05)

Dado que el coef = -0.350547 < 0, se interpreta que el estar en el departamento de ventas (no estar en el departamento de investigación) aumenta el riesgo de renunciar.

In [67]:
p = (1- cox1.hazard_ratios_['dep_Investigación']).round(4)
p

0.2957

El riesgo que los empleados renuncien se reduce en un 29.57% al pertenecer al departamento de Investigación respecto a estar en el departamento de Ventas.

**sal_bajo** (P-value < 0.005 < alfa = 0.05)

Dado que el coef = 1.538901 > 0, el tener un salario de categoría bajo aumenta el riesgo de renunciar, comparado a aquellos empleados quienes tienen un salario alto.

In [69]:
p = (cox1.hazard_ratios_['sal_bajo'] - 1).round(4)
p

3.6595

El riesgo que los empleados renuncien aumenta en un 365.95% al tener un salario bajo, comparado a quienes tienen un salario alto.

**sal_medio** (P-value < 0.005 < alfa = 0.05)

Dado que el coef =  1.147612 > 0, el tener un salario de categoría medio aumenta el riesgo que los empleados renuncien, comparado a quienes tienen un salario alto.

In [71]:
p = (cox1.hazard_ratios_['sal_medio'] - 1).round(4)
p

2.1507

El riesgo que los empleados renuncien aumenta un 215.07% al tener un salario medio, comparado a los empleados con un salario alto

#### ¿Qué recomendación podría hacerse a la empresa?

El buscar el equilibrio entre los salarios es crucial para reducir la cantidad de renuncias, pues al compararlo a aquellos salarios considerados altos, el riesgo de renunciar por tener un salario bajo o medio aumenta demasiado.

Dado que el riesgo de renunciar reduce al aumentar la cantidad de proyectos asignados, así como al haber sido ascendido durante los últimos 5 años, se puede notar que los empleados no renuncian tanto al contar con cierta "motivación", por lo que un nuevo sistema de organización en el que todos tengan la oportunidad de escalar jerárquicamente, así como un sistema de trabajo por proyectos podría mejorar a reducir la cantidad de renuncias.

El nivel de satisfacción hace notar que, a mayor satisfacción de los empleados, menor probabilidad que renuncien, por lo que, la empresa podría buscar la forma de hacer sentir más felices a sus empleados, ya sea con mejores instalaciones o equipamiento, actividades fuera de horario de trabajo que incentiven la comunicación entre todos, así como el networking entre las distintas áreas laborales de la empresa.

El indicador de accidente puede ser confuso, y necesitaría más contexto para poder hacer una recomendación de este, pues terminaría de ser raro decir que a la empresa le conviene "que sus empleados se accidenten dentro de la empresa para que no quieran renunciar".

#### Probar el modelo de Cox si cumple el supuesto de riesgos proporcionales

In [72]:
cox1.check_assumptions(data, p_value_threshold=0.05)

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
         test_name = proportional_hazard_test

---
                                 test_statistic      p  -log2(p)
accidente                  km              0.03   0.87      0.21
                           rank            1.14   0.29      1.81
ascenso_5years             km              3.66   0.06      4.17
                           rank            6.71   0.01      6.71
dep_Investigación          km              1.04   0.31      1.70
                           rank            4.36   0.04      4.76
dep_administración         km              3.66   0.06      4.17
                           rank            1.53   0.22      2.21
dep_contabilidad           km              0.52   0.47      1.09
                           rank            0.71   0.40      1.33
dep_desarrollo de producto km              0.07   0.79      0.34
                           rank            1.10   0.29      1.77
dep_informática            km              0.08   0.77      0.37
                           rank            2.59   0.11      3.22
dep_mantenimiento          km              0.31   0.58      0.79
                           rank            0.91   0.34      1.56
dep_mercadotecnia          km              1.24   0.27      1.91
                           rank            1.31   0.25      1.99
dep_recursos humanos       km              0.00   0.99      0.01
                           rank            0.00   1.00      0.00
dep_soporte                km              2.68   0.10      3.30
                           rank            3.25   0.07      3.81
evaluacion                 km            443.05 <0.005    324.32
                           rank          427.15 <0.005    312.82
promedio_horas_mes         km            270.84 <0.005    199.74
                           rank          307.41 <0.005    226.21
proyectos                  km            574.13 <0.005    419.05
                           rank          684.93 <0.005    499.11
sal_bajo                   km              0.37   0.54      0.88
                           rank            0.00   0.96      0.06
sal_medio                  km              0.16   0.69      0.53
                           rank            0.10   0.75      0.42
satisfaccion               km           1257.74 <0.005    912.74
                           rank          845.24 <0.005    614.90



1. Variable 'satisfaccion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'satisfaccion' might be incorrect. That is, there
may be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'satisfaccion' using pd.cut, and then specify it in
`strata=['satisfaccion', ...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'evaluacion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'evaluacion' might be incorrect. That is, there may
be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to 

[]

Este modelo no cumple el supuesto de riesgo proporcional constante, debemos utilizar otro modelo (AFT, modelo de riesgo acelerado).

#### Modelo AFT

In [73]:
mexpo = ExponentialFitter().fit(data['semestre'], data['renunciar'])
mweib = WeibullFitter().fit(data['semestre'], data['renunciar'])
mlogn = LogNormalFitter().fit(data['semestre'], data['renunciar'])

In [74]:
##Criterio AIC (menor  = mejor)
print(f'Exponencial: {mexpo.AIC_.round(2)}, Weibull: {mweib.AIC_.round(2)}, LogN: {mlogn.AIC_.round(2)}')

Exponencial: 15965.46, Weibull: 12511.51, LogN: 11587.92


El mejor modelo es el LogN, es el utilizar en el AFT

In [75]:
from lifelines import LogNormalAFTFitter

In [76]:
lnaft = LogNormalAFTFitter().fit(data, 'semestre', 'renunciar')
lnaft.print_summary()

<lifelines.LogNormalAFTFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
   number of observations = 11991
number of events observed = 1991
           log-likelihood = -4876.57
         time fit was run = 2023-09-24 21:38:56 UTC

---
                                    coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param  covariate                                                                                                                          
mu_    accidente                    0.26       1.30       0.02             0.22             0.30                 1.25                 1.35
       ascenso_5years               0.37       1.44       0.06             0.24             0.49                 1.27                 1.63
       dep_Investigación            0.06       1.06       0.03             0.00             0.11                 1.00                 1.12
       dep_administración           0.11       1.11       0.03             0.04             0.17                 1.04                 1.18
       dep_contabilidad             0.01       1.01       0.03            -0.04             0.06                 0.96                 1.07
       dep_desarrollo de producto   0.01       1.01       0.02            -0.04             0.05                 0.96                 1.06
       dep_informática              0.01       1.01       0.02            -0.03             0.05                 0.97                 1.05
       dep_mantenimiento           -0.02       0.98       0.02            -0.05             0.01                 0.95                 1.01
       dep_mercadotecnia            0.02       1.02       0.03            -0.03             0.07                 0.97                 1.07
       dep_recursos humanos        -0.03       0.97       0.03            -0.08             0.02                 0.92                 1.02
       dep_soporte                 -0.03       0.97       0.02            -0.06             0.01                 0.94                 1.01
       evaluacion                   0.01       1.01       0.00            -0.00             0.01                 1.00                 1.01
       promedio_horas_mes          -0.00       1.00       0.00            -0.00            -0.00                 1.00                 1.00
       proyectos                    0.10       1.10       0.01             0.09             0.11                 1.09                 1.11
       sal_bajo                    -0.37       0.69       0.03            -0.43            -0.31                 0.65                 0.73
       sal_medio                   -0.29       0.75       0.03            -0.34            -0.23                 0.71                 0.80
       satisfaccion                 0.07       1.07       0.00             0.06             0.07                 1.06                 1.07
       Intercept                    1.28       3.61       0.04             1.21             1.36                 3.35                 3.90
sigma_ Intercept                   -1.11       0.33       0.02            -1.14            -1.08                 0.32                 0.34

                                    cmp to      z      p   -log2(p)
param  covariate                                                   
mu_    accidente                      0.00  12.98 <0.005     125.59
       ascenso_5years                 0.00   5.80 <0.005      27.19
       dep_Investigación              0.00   2.13   0.03       4.92
       dep_administración             0.00   3.22 <0.005       9.60
       dep_contabilidad               0.00   0.55   0.58       0.79
       dep_desarrollo de producto     0.00   0.22   0.82       0.28
       dep_informática                0.00   0.51   0.61       0.71
       dep_mantenimiento              0.00  -1.25   0.21       2.25
       dep_mercadotecnia              0.00   0.86   0.39

In [77]:
## columna coeficientes
lnaft.params_

param   covariate                 
mu_     accidente                     0.263049
        ascenso_5years                0.365026
        dep_Investigación             0.057362
        dep_administración            0.105330
        dep_contabilidad              0.014074
        dep_desarrollo de producto    0.005554
        dep_informática               0.010962
        dep_mantenimiento            -0.020279
        dep_mercadotecnia             0.021780
        dep_recursos humanos         -0.028864
        dep_soporte                  -0.025662
        evaluacion                    0.005721
        promedio_horas_mes           -0.000309
        proyectos                     0.096127
        sal_bajo                     -0.369968
        sal_medio                    -0.286107
        satisfaccion                  0.067044
        Intercept                     1.284409
sigma_  Intercept                    -1.113762
dtype: float64

In [80]:
## columna exp coef
summary = lnaft.summary
ex = summary['exp(coef)']
ex

param   covariate                 
mu_     accidente                     1.300891
        ascenso_5years                1.440551
        dep_Investigación             1.059039
        dep_administración            1.111077
        dep_contabilidad              1.014173
        dep_desarrollo de producto    1.005569
        dep_informática               1.011022
        dep_mantenimiento             0.979925
        dep_mercadotecnia             1.022019
        dep_recursos humanos          0.971548
        dep_soporte                   0.974664
        evaluacion                    1.005737
        promedio_horas_mes            0.999691
        proyectos                     1.100899
        sal_bajo                      0.690757
        sal_medio                     0.751182
        satisfaccion                  1.069342
        Intercept                     3.612532
sigma_  Intercept                     0.328321
Name: exp(coef), dtype: float64

##### Interpretar coeficientes de modelo AFT

##### Interpretación de coeficientes

Comenzando por establecer las hipótesis:

$H_o$: La variable no afecta

$H_a$: La variable tiene un efecto sobre que los empleados renuncien 

Estableciendo un valor alfa = 0.05

Las variables que no tienen algún efecto sobre que los empleados renuncien son:

1. **dep_contabilidad** (P-value = 0.58 > alfa = 0.05)
2. **dep_desarrollo de producto** (P-value = 0.82 > alfa = 0.05)
3. **dep_informática** (P-value = 0.61 > alfa = 0.05)
4. **dep_mantenimiento** (P-value = 0.21 > alfa = 0.05)
5. **dep_mercadotecnia** (P-value = 0.39 > alfa = 0.05)
6. **dep_recursos humanos** (P-value = 0.25 > alfa = 0.05)
7. **dep_soporte** (P-value = 0.14 > alfa = 0.05)
8. **evaluación** (P-value = 0.14 > alfa = 0.05)

Ahora, comenzando para aquellas que sí son significativas y tienen efecto sobre que los empleados renuncien:

**accidente**  (P-value < 0.005 < alfa = 0.05)

Dado que el coef = 0.263049 > 0, se interpreta que sufrir un accidente dentro de la empresa aumenta el riesgo de renunciar, comparado a los empleados que no han sufrido un accidente dentro de la empresa.

In [82]:
p = (1.300891-1)
p

0.300891

El riesgo que los empleados renuncien aumenta un 30.08% si el empleado sufrió un accidente dentro de la empresa, comparado a los empleados que no han sufrido un accidente dentro de la empresa.

**ascenso_5years** (P-value < 0.005 < alfa = 0.05)

Dado que el coef = 0.365026 > 0, se interpreta que el haber ascendido durante los últimos 5 años aumenta el riesgo de renunciar.

In [83]:
p = 1.440551 -1
p

0.4405509999999999

El riesgo que los empleados renuncien aumenta un 44.05% si el empleado fue ascendido durante los últimos 5 años, comparado a los empleados que no han sido ascendido durante los últimos 5 años.

**dep_Investigación** (P-value = 0.03 < alfa = 0.05)

Dado que el coef = 0.057362 > 0, se interpreta que estar en el departamento de Investigación aumenta el riesgo de renunciar, sobre los empleados que están en el departamento de Ventas.

In [86]:
p= (1.059039 - 1)
p

0.059039000000000064

El riesgo que los empleados renuncien aumenta un 5.9% si el empleado está en el departamento de Investigación, sobre los empleados que están en el departamento de Ventas.

**dep_Administración** (P-value < 0.005 < alfa = 0.05)

Dado que el coef = 0.105330 >0, estar en el departamento de Administración implica un  aumento en el riesgo de renunciar, sobre los empleados que están en el departamento de Ventas.

In [87]:
p = 1.111077 - 1
p

0.11107700000000009

El riesgo que los empleados auemnta un 11.1% si el empleado está en el departamento de Administración, sobre los empleados que están en el departamento de ventas.

**proedio_horas_mes** (P-value < 0.005 < alfa = 0.05)

Dado que el coef = -0.000309 < 0, una menor cantidad de horas promedio mensuales aumenta el riesgo de renunciar de parte de los empleados.

In [88]:
p = 1 - 0.999691
p

0.0003090000000000037

El riesgo que los empleados renuncien se reduce un 0.03% por cada hora promedio mensual.

**proyectos** (P-value < 0.005 < alfa = 0.05)

Dado que el coef = 0.096127 > 0, se interpreta que una mayor cantidad de proyectos aumenta el riesgo de renunciar de parte de los empleados.


In [89]:
p = 1.100899 - 1
p

0.10089900000000007

El riesgo que los empleados renuncien aumenta un 10.08% por cada proyecto asignado al empleado.

**sal_bajo** (P-value < 0.005 < alfa = 0.05)

Dado que el coef = -0.369968 < 0, el tener un salario categorizado como bajo reduce el riesgo que los empleados renuncien, comparado a aquellos con un salario categorizado alto.

In [92]:
p = 1 - 0.690757
p

0.30924300000000005

El riesgo que los empleados renuncien se reduce un 30.92% al tener un salario bajo, comparado a aquellos con un salario algo.

**sal_medio** (P-value < 0.005 < alfa = 0.05)

Dado que el coef = -0.286107 < 0, el tener un salario categorizado como medio reduce el riesgo que los empleados renuncien, comparado a aquellos con un salario categorizado alto.

In [93]:
p = 1 - 0.751182
p

0.24881799999999998

El riesgo que los empleados renuncien se reduce un 24.88% al tener un salario medio, comparado a aquellos con un salario bajo.